In [1]:
from sklearn import datasets, tree, metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
import numpy as np
import xgboost as xgb

Заведите массив для объектов DecisionTreeRegressor (будем их использовать в качестве базовых алгоритмов) и для вещественных чисел (это будут коэффициенты перед базовыми алгоритмами). В цикле обучите последовательно 50 решающих деревьев с параметрами max_depth=5 и random_state=42 (остальные параметры - по умолчанию). В бустинге зачастую используются сотни и тысячи деревьев, но мы ограничимся 50, чтобы алгоритм работал быстрее, и его было проще отлаживать (т.к. цель задания разобраться, как работает метод). Каждое дерево должно обучаться на одном и том же множестве объектов, но ответы, которые учится прогнозировать дерево, будут меняться в соответствие с полученным в задании 1 правилом. Попробуйте для начала всегда брать коэффициент равным 0.9. Обычно оправдано выбирать коэффициент значительно меньшим - порядка 0.05 или 0.1, но т.к. в нашем учебном примере на стандартном датасете будет всего 50 деревьев, возьмем для начала шаг побольше.

В процессе реализации обучения вам потребуется функция, которая будет вычислять прогноз построенной на данный момент композиции деревьев на выборке X:


(считаем, что base_algorithms_list - список с базовыми алгоритмами, coefficients_list - список с коэффициентами перед алгоритмами)

Эта же функция поможет вам получить прогноз на контрольной выборке и оценить качество работы вашего алгоритма с помощью mean_squared_error в sklearn.metrics. Возведите результат в степень 0.5, чтобы получить RMSE. Полученное значение RMSE — ответ в пункте 2.

In [2]:
data = datasets.load_boston()
X = data.data
y = data.target

In [3]:
(X_train, 
 X_test, 
 y_train, y_test) = train_test_split(X, y, 
                                     test_size=0.25, 
                                     random_state=0)

print X_train.shape
print X_test.shape
print y_train.shape
print y_test.shape

(379L, 13L)
(127L, 13L)
(379L,)
(127L,)


Задание 1

Как вы уже знаете из лекций, бустинг - это метод построения композиций базовых алгоритмов с помощью последовательного добавления к текущей композиции нового алгоритма с некоторым коэффициентом. Градиентный бустинг обучает каждый новый алгоритм так, чтобы он приближал антиградиент ошибки по ответам композиции на обучающей выборке. Аналогично минимизации функций методом градиентного спуска, в градиентном бустинге мы подправляем композицию, изменяя алгоритм в направлении антиградиента ошибки. Воспользуйтесь формулой из лекций, задающей ответы на обучающей выборке, на которые нужно обучать новый алгоритм (фактически это лишь чуть более подробно расписанный градиент от ошибки), и получите частный ее случай, если функция потерь L - квадрат отклонения ответа композиции a(x) от правильного ответа y на данном x.

Если вы давно не считали производную самостоятельно, вам поможет таблица производных элементарных функций (которую несложно найти в интернете) и правило дифференцирования сложной функции. После дифференцирования квадрата у вас возникнет множитель 2 — т.к. нам все равно предстоит выбирать коэффициент, с которым будет добавлен новый базовый алгоритм, проигноируйте этот множитель при дальнейшем построении алгоритма.

In [4]:
def get_delta (y, a):
    y = np.array(y)
    a = np.array (a)
    return -(a - y)



In [5]:
def gbm_predict(X):

    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]


In [6]:
base_algorithms_list = []
coefficients_list = np.array([0.9]*50)

y_for_fit = y_train

for i in range(50):
 
    tree_regressor = tree.DecisionTreeRegressor (max_depth=5, random_state=42)
    tree_regressor.fit(X_train, y_for_fit)    
    
    base_algorithms_list.append(tree_regressor)

    y_for_fit = get_delta (y_train, gbm_predict(X_train))
    

    

In [7]:
y_predicted = gbm_predict(X_test)
MSE = metrics.mean_squared_error (y_test, y_predicted)
RMSE = MSE**0.5
print RMSE

with open('C2W4 task2-2.txt', 'w') as f:
    f.write(str(RMSE))

5.204116464919526


Задание 3

Вас может также беспокоить, что при движении с постоянным шагом вблизи минимума ошибки ответы на обучающей выборке меняются слишком резко, перескакивая через минимум. Попробуйте уменьшать вес перед каждым алгоритмом с каждой следующей итерацией по формуле 0.9 / (1.0 + i), где i - номер итерации (от 0 до 49). Используйте качество работы алгоритма как ответ в пункте 3. В реальности часто применяется следующая стратегия выбора шага: как только выбран алгоритм, подберем коэффициент перед ним численным методом оптимизации таким образом, чтобы отклонение от правильных ответов было минимальным. Мы не будем предлагать вам реализовать это для выполнения задания, но рекомендуем попробовать разобраться с такой стратегией и реализовать ее при случае для себя.

In [8]:
base_algorithms_list = []
coefficients_list = np.array([0.9/(1.0 + x) for x in range(50)])

y_for_fit = y_train

for i in range(50):
 
    tree_regressor = tree.DecisionTreeRegressor (max_depth=5, random_state=42)
    tree_regressor.fit(X_train, y_for_fit)    
    
    base_algorithms_list.append(tree_regressor)

    y_for_fit = get_delta (y_train, gbm_predict(X_train))

In [9]:
y_predicted = gbm_predict(X_test)
MSE = metrics.mean_squared_error (y_test, y_predicted)
RMSE = MSE**0.5
print RMSE

with open('C2W4 task2-3.txt', 'w') as f:
    f.write(str(RMSE))

4.968891286623323


Задание 4

Реализованный вами метод - градиентный бустинг над деревьями - очень популярен в машинном обучении. Он представлен как в самой библиотеке sklearn, так и в сторонней библиотеке XGBoost, которая имеет свой питоновский интерфейс. На практике XGBoost работает заметно лучше GradientBoostingRegressor из sklearn, но для этого задания вы можете использовать любую реализацию. Исследуйте, переобучается ли градиентный бустинг с ростом числа итераций (и подумайте, почему), а также с ростом глубины деревьев. На основе наблюдений выпишите через пробел номера правильных из приведенных ниже утверждений в порядке возрастания номера (это будет ответ в п.4):

1) С увеличением числа деревьев, начиная с некоторого момента, качество работы градиентного бустинга не меняется существенно.

2) С увеличением числа деревьев, начиная с некоторого момента, градиентный бустинг начинает переобучаться.

3) С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга на тестовой выборке начинает ухудшаться

4) С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга перестает существенно изменяться

In [10]:
def make_xgb_regression (max_depth, n_estimators):
#    print 'max_depth = ', max_depth
#    print 'n_estimators = ', n_estimators
    estimator = xgb.XGBRegressor(max_depth=max_depth, n_estimators=n_estimators, learning_rate=1)
    estimator.fit(X_train, y_train)  
    y_predicted = estimator.predict(X_test)
    
    MSE = metrics.mean_squared_error (y_test, y_predicted)
    RMSE = MSE**0.5
    print 'RMSE = ', RMSE
    
#    score = cross_val_score(estimator, X, y, cv = 3, scoring='mean_squared_error') 
#    print 'score = ', score
    
    return RMSE

In [11]:
# Меняем количество деревьев
make_xgb_regression (5, 2)
make_xgb_regression (5, 5)
make_xgb_regression (5, 10)
make_xgb_regression (5, 20)
make_xgb_regression (5, 50)
make_xgb_regression (5, 70)
make_xgb_regression (5, 100)
make_xgb_regression (5, 150)
make_xgb_regression (5, 200)
make_xgb_regression (5, 300)

[12:59:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE =  4.796254959586402
[12:59:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE =  4.4518514709595065
[12:59:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE =  4.48026978029052
[12:59:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE =  4.513987522566191
[12:59:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE =  4.5152472495166105
[12:59:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE =  4.514845238533444
[12:59:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE =  4.514853087928584
[12:5

4.514853087928584

In [12]:
# Меняем глубину деревьев
print make_xgb_regression (2, 50)
print make_xgb_regression (5, 50)
print make_xgb_regression (7, 50)
print make_xgb_regression (10, 50)
print make_xgb_regression (15, 50)
print make_xgb_regression (20, 50)
print make_xgb_regression (30, 50)
print make_xgb_regression (100, 50)

[12:59:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE =  4.931641506490305
4.931641506490305
[12:59:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE =  4.5152472495166105
4.5152472495166105
[12:59:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE =  4.744766489942703
4.744766489942703
[12:59:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE =  5.036030962232374
5.036030962232374
[12:59:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE =  4.719567439574956
4.719567439574956
[12:59:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE =  4.761684241454601
4.761684241454601
[12:59:15] WARNING: src/objective/regr

In [13]:
with open('C2W4 task2-4.txt', 'w') as f:
    f.write ('1 4')

Задание 5

Сравните получаемое с помощью градиентного бустинга качество с качеством работы линейной регрессии. Для этого обучите LinearRegression из sklearn.linear_model (с параметрами по умолчанию) на обучающей выборке и оцените для прогнозов полученного алгоритма на тестовой выборке RMSE. Полученное качество - ответ в пункте 5. В данном примере качество работы простой модели должно было оказаться хуже, но не стоит забывать, что так бывает не всегда. В заданиях к этому курсу вы еще встретите пример обратной ситуации.

In [14]:
X_train = X[:379, :]
X_test = X[379:, :]
y_train = y[:379]
y_test = y[379:]

print y.shape
print y_train.shape
print y_test.shape

estimator = LinearRegression()
estimator.fit(X_train, y_train)
y_predicted = estimator.predict(X_test)

MSE = metrics.mean_squared_error (y_test, y_predicted)
RMSE = np.sqrt(MSE)
print 'RMSE = ', RMSE


(506L,)
(379L,)
(127L,)
RMSE =  8.254979753549161


In [15]:
with open('C2W4 task2-5.txt', 'w') as f:
    f.write (str(RMSE))